## Prepare dataset

In [1]:
# Preparation of the YOLO dataset

# Import necessary modules
import random  # For generating random numbers
import os  # For creating directories and interacting with the file system
import shutil  # For high-level file operations like copying and removing files
import csv  # For reading and writing CSV files

# Set the seed for random number generation to ensure reproducibility
random.seed(1)

# Number of training images
train_n = 800

# Generate a list of random indices for training images
train_index = random.sample(range(1, 1000 + 1), train_n)

# Remove the existing 'yolo_dataset' directory if it exists
if os.path.isdir('yolo_dataset/'):
    shutil.rmtree('yolo_dataset/')

# Create necessary directories for the YOLO dataset
os.makedirs('yolo_dataset')
os.makedirs('yolo_dataset/images')
os.makedirs('yolo_dataset/labels')
os.makedirs('yolo_dataset/images/train')
os.makedirs('yolo_dataset/images/val')
os.makedirs('yolo_dataset/labels/train')
os.makedirs('yolo_dataset/labels/val')

# Create a YAML file for the YOLO dataset configuration
dataset_yaml = f"""names:
- circle
- triangle
- rectangle
nc: 3
path: {os.getcwd()}/yolo_dataset
train: images/train/
val: images/val/
"""

# Write the YAML configuration to a file
with open('yolo_dataset/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

# Copy images to their respective training and validation directories
for i in range(1, 1001):
    target = 'train' if i in train_index else 'val'
    shutil.copyfile(f'result/random_shapes_{i}.png', f'yolo_dataset/images/{target}/random_shapes_{i}.png')

# Read the annotations from the CSV file
with open('annotations.csv', 'r') as f:
    data = []
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        data.append(row)

# Remove the header row from the data
data.pop(0)

# Define a mapping from shape names to class indices
class_index = {
    "circle": 0,
    "triangle": 1,
    "rectangle": 2
}

# Create label files for each image in YOLO format
for i in range(1, 1001):
    target = 'train' if i in train_index else 'val'
    shapes = [[x[0], int(x[1]), int(x[2]), int(x[3]), int(x[4]), x[5]] for x in data if x[0] == f'result/random_shapes_{i}.png']
    with open(f'yolo_dataset/labels/{target}/random_shapes_{i}.txt', 'w') as f:
        for shape in shapes:
            # Calculate YOLO format coordinates
            x_center = 0.5 * (shape[1] + shape[3]) / 100
            y_center = 0.5 * (shape[2] + shape[4]) / 100
            w = (shape[3] - shape[1]) / 100
            h = (shape[4] - shape[2]) / 100
            # Write the annotation to the file
            f.write(f'{class_index[shape[5]]} {x_center} {y_center} {w} {h}\n')

In [2]:
shape

['result/random_shapes_1000.png', 56, 72, 77, 93, 'circle']

In [3]:
# Training the YOLO model

# Import YOLO from the ultralytics library
from ultralytics import YOLO

# Load the YOLO model with a pre-trained weights file
model = YOLO("yolov8n.pt")

# Train the model using the prepared dataset
results = model.train(data="./yolo_dataset/dataset.yaml", verbose=True, epochs=100, imgsz=100, device=[0], name='yolo', save_period=5)

New https://pypi.org/project/ultralytics/8.2.35 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.32  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./yolo_dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=100, save=True, save_period=5, cache=False, device=[0], workers=8, project=None, name=yolo, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

train: Scanning G:\shapes\yolo_dataset\labels\train... 800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 800/800 [

train: New cache created: G:\shapes\yolo_dataset\labels\train.cache



val: Scanning G:\shapes\yolo_dataset\labels\val... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:0

val: New cache created: G:\shapes\yolo_dataset\labels\val.cache


...

    100/100     0.191G     0.4454     0.3377     0.8427         33        128: 100%|██████████| 50/50 [00:03<00:00, 12.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<0

                   all        200        402      0.967      0.929      0.974      0.873



100 epochs completed in 0.147 hours.
Optimizer stripped from runs\detect\yolo\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolo\weights\best.pt, 6.2MB

Validating runs\detect\yolo\weights\best.pt...
Ultralytics YOLOv8.2.32  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<0


                   all        200        402      0.963      0.931      0.974      0.881
                circle        109        140      0.999      0.993      0.995       0.94
              triangle        101        121      0.971      0.821      0.936      0.779
             rectangle        114        141       0.92       0.98      0.992      0.924
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\yolo
